# 📊 Mejoras en Análisis Demográfico de Tendencias de Nombres

> **Fecha:** Junio 2025
> **Autor:** Data Science Portfolio

Este notebook amplía el análisis demográfico de tendencias de nombres en Estados Unidos desde 1910 hasta 2013, utilizando datos de BigQuery. El objetivo es generar visualizaciones avanzadas y explicaciones detalladas que serán incorporadas a la aplicación Streamlit.

## 🎯 Objetivos:
- Crear visualizaciones más detalladas y atractivas
- Proporcionar explicaciones contextuales de tendencias y patrones
- Analizar aspectos socioculturales que influyen en las tendencias de nombres
- Generar insights que enriquezcan la aplicación Streamlit

## 1. Importación de Librerías

Primero, importamos todas las librerías necesarias para el análisis y la visualización.

In [1]:
# Librerías básicas para manejo de datos
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.ticker as mtick

# Configuración visual y de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['legend.title_fontsize'] = 12

# Para ignorar advertencias irrelevantes
import warnings
warnings.filterwarnings('ignore')

# Para guardar las visualizaciones
import os
from pathlib import Path

# Configuración de seeds para reproducibilidad
np.random.seed(42)

## 2. Carga y Exploración de Datos Demográficos

En esta sección, cargaremos los datos de nombres históricos desde 1910 hasta 2013 en Estados Unidos. Normalmente estos datos provienen de BigQuery, pero para este notebook utilizaremos datos sintéticos representativos que simulan los resultados de la consulta.

In [2]:
# Función para generar datos sintéticos de nombres
def generate_synthetic_names_data(start_year=1910, end_year=2013):
    """
    Genera datos sintéticos que simulan resultados de consultas a BigQuery sobre nombres
    en EE.UU. desde start_year hasta end_year.
    """
    # Lista de nombres populares por género para crear nuestros datos
    popular_male_names = [
        'James', 'John', 'Robert', 'Michael', 'William', 'David', 'Joseph', 'Charles', 
        'Thomas', 'Christopher', 'Daniel', 'Matthew', 'Anthony', 'Donald', 'Mark',
        'Paul', 'Steven', 'Andrew', 'Kenneth', 'George', 'Joshua', 'Kevin', 'Brian',
        'Edward', 'Ronald', 'Timothy', 'Jason', 'Jeffrey', 'Ryan', 'Jacob', 'Gary',
        'Nicholas', 'Eric', 'Stephen', 'Jonathan', 'Larry', 'Justin', 'Scott', 'Brandon',
        'Frank', 'Benjamin', 'Gregory', 'Samuel', 'Raymond', 'Patrick', 'Alexander', 
        'Jack', 'Dennis', 'Jerry', 'Tyler', 'Aaron', 'Henry', 'Douglas', 'Jose', 'Peter',
        'Adam', 'Nathan', 'Zachary', 'Walter', 'Kyle', 'Harold', 'Carl', 'Jeremy', 'Gerald',
        'Keith', 'Roger', 'Arthur', 'Terry', 'Lawrence', 'Sean', 'Christian', 'Ethan',
        'Austin', 'Joe', 'Albert', 'Jesse', 'Willie', 'Billy', 'Bryan', 'Bruce', 'Jordan',
        'Ralph', 'Roy', 'Noah', 'Dylan', 'Eugene', 'Wayne', 'Alan', 'Juan', 'Louis', 'Russell',
        'Gabriel', 'Randy', 'Philip', 'Harry', 'Vincent', 'Bobby', 'Johnny', 'Logan'
    ]
    
    popular_female_names = [
        'Mary', 'Patricia', 'Jennifer', 'Linda', 'Elizabeth', 'Barbara', 'Susan', 'Jessica',
        'Sarah', 'Karen', 'Nancy', 'Margaret', 'Lisa', 'Betty', 'Dorothy', 'Sandra', 'Ashley',
        'Kimberly', 'Donna', 'Emily', 'Michelle', 'Carol', 'Amanda', 'Melissa', 'Deborah',
        'Stephanie', 'Rebecca', 'Laura', 'Sharon', 'Cynthia', 'Kathleen', 'Helen', 'Amy',
        'Shirley', 'Angela', 'Anna', 'Ruth', 'Brenda', 'Pamela', 'Nicole', 'Katherine',
        'Samantha', 'Christine', 'Catherine', 'Virginia', 'Debra', 'Rachel', 'Janet',
        'Emma', 'Carolyn', 'Maria', 'Heather', 'Diane', 'Julie', 'Joyce', 'Victoria',
        'Kelly', 'Christina', 'Lauren', 'Joan', 'Evelyn', 'Olivia', 'Judith', 'Megan',
        'Cheryl', 'Martha', 'Andrea', 'Frances', 'Hannah', 'Jacqueline', 'Ann', 'Gloria',
        'Jean', 'Kathryn', 'Alice', 'Teresa', 'Sara', 'Janice', 'Doris', 'Madison', 'Julia',
        'Grace', 'Judy', 'Abigail', 'Marie', 'Denise', 'Beverly', 'Amber', 'Theresa', 'Marilyn',
        'Danielle', 'Diana', 'Brittany', 'Natalie', 'Sophia', 'Rose', 'Isabella', 'Alexis'
    ]
    
    years = list(range(start_year, end_year + 1))
    
    # Generar datos para nombres masculinos
    male_records = []
    for name in popular_male_names:
        # Simular un patrón de popularidad que varía con el tiempo
        base_count = np.random.randint(500, 10000)
        peak_year = np.random.randint(start_year, end_year)
        peak_width = np.random.randint(5, 30)
        
        for year in years:
            # Crear un patrón de campana para la popularidad
            time_factor = np.exp(-0.5 * ((year - peak_year) / peak_width) ** 2)
            count = int(base_count * time_factor * (0.8 + 0.4 * np.random.random()))
            
            # Añadir algo de ruido y tendencia
            trend_factor = 1 + 0.001 * (year - start_year) * (np.random.random() - 0.5)
            count = max(1, int(count * trend_factor * (0.9 + 0.2 * np.random.random())))
            
            male_records.append({
                'name': name,
                'year': year,
                'gender': 'M',
                'count': count
            })
    
    # Generar datos para nombres femeninos
    female_records = []
    for name in popular_female_names:
        base_count = np.random.randint(500, 10000)
        peak_year = np.random.randint(start_year, end_year)
        peak_width = np.random.randint(5, 30)
        
        for year in years:
            time_factor = np.exp(-0.5 * ((year - peak_year) / peak_width) ** 2)
            count = int(base_count * time_factor * (0.8 + 0.4 * np.random.random()))
            
            trend_factor = 1 + 0.001 * (year - start_year) * (np.random.random() - 0.5)
            count = max(1, int(count * trend_factor * (0.9 + 0.2 * np.random.random())))
            
            female_records.append({
                'name': name,
                'year': year,
                'gender': 'F',
                'count': count
            })
    
    # Combinar los registros
    all_records = male_records + female_records
    
    # Crear DataFrame
    df = pd.DataFrame(all_records)
    
    return df

# Generar datos sintéticos
names_df = generate_synthetic_names_data()

# Mostrar las primeras filas
names_df.head()

,name,year,gender,count
0,James,1910,M,215
1,James,1911,M,225
2,James,1912,M,262
3,James,1913,M,357
4,James,1914,M,305


In [3]:
# Información básica del dataset
print(f"Dimensiones del dataset: {names_df.shape}")
print(f"Período temporal cubierto: {names_df['year'].min()} - {names_df['year'].max()}")
print(f"Número total de nombres únicos: {names_df['name'].nunique()}")
print(f"Distribución por género:")
print(names_df.groupby('gender')['name'].nunique())

# Calcular algunos estadísticos básicos por género
gender_stats = names_df.groupby(['gender', 'year'])['count'].sum().reset_index()
gender_stats_wide = gender_stats.pivot(index='year', columns='gender', values='count')

# Muestra de estadísticas descriptivas
names_df.describe()

Dimensiones del dataset: (20488, 4)
Período temporal cubierto: 1910 - 2013
Número total de nombres únicos: 197
Distribución por género:
gender
F    98
M    99
Name: name, dtype: int64


,year,count
count,20488.000000,20488.000000
mean,1961.500000,1765.267083
std,30.021559,2363.148279
min,1910.000000,1.000000
25%,1935.750000,17.000000
50%,1961.500000,644.000000
75%,1987.250000,2703.000000
max,2013.000000,12284.000000


## 3. Visualizaciones Avanzadas y Análisis Demográfico

En esta sección, crearemos visualizaciones detalladas para explorar diferentes aspectos de los datos de nombres a lo largo del tiempo, agregando explicaciones contextuales e insights.

### 3.1 Tendencias Históricas de Nombres por Género (1910-2013)

Esta visualización muestra la evolución en el número de bebés registrados por género a lo largo del tiempo. Nos permite observar cambios demográficos importantes, patrones generacionales y posibles influencias de eventos históricos en las tasas de natalidad.

In [4]:
# Agregar datos por género y año
gender_year_counts = names_df.groupby(['year', 'gender'])['count'].sum().reset_index()

# Crear la figura con Plotly para mejor interactividad
fig = px.line(gender_year_counts, x="year", y="count", color="gender", 
              color_discrete_map={"M": "#1f77b4", "F": "#e377c2"},
              labels={"count": "Número de nacimientos registrados", 
                      "year": "Año", 
                      "gender": "Género"},
              title="Evolución histórica de nacimientos registrados por género (1910-2013)")

# Agregar eventos históricos relevantes como anotaciones
historical_events = [
    {"year": 1918, "event": "Fin de la Primera Guerra Mundial", "y": gender_year_counts[gender_year_counts.year == 1918]['count'].max() + 200000},
    {"year": 1929, "event": "Gran Depresión", "y": gender_year_counts[gender_year_counts.year == 1929]['count'].max() + 200000},
    {"year": 1945, "event": "Fin de la Segunda Guerra Mundial", "y": gender_year_counts[gender_year_counts.year == 1945]['count'].max() + 200000},
    {"year": 1946, "event": "Inicio del Baby Boom", "y": gender_year_counts[gender_year_counts.year == 1946]['count'].max() + 200000},
    {"year": 1964, "event": "Fin del Baby Boom", "y": gender_year_counts[gender_year_counts.year == 1964]['count'].max() + 200000},
    {"year": 1973, "event": "Crisis del petróleo", "y": gender_year_counts[gender_year_counts.year == 1973]['count'].max() + 200000},
    {"year": 1991, "event": "Fin de la Guerra Fría", "y": gender_year_counts[gender_year_counts.year == 1991]['count'].max() + 200000},
    {"year": 2007, "event": "Inicio de la Gran Recesión", "y": gender_year_counts[gender_year_counts.year == 2007]['count'].max() + 200000},
]

# Añadir líneas verticales para eventos importantes
for event in historical_events:
    fig.add_vline(x=event["year"], line_dash="dash", line_width=1, 
                  annotation_text=event["event"], 
                  annotation_position="top right")

# Mejorar el diseño
fig.update_layout(
    height=600,
    legend_title_text="Género",
    hovermode="x unified",
    template="plotly_white",
    font=dict(size=12),
    xaxis=dict(
        tickmode='linear',
        dtick=10,  # Tick cada 10 años
        tickangle=45
    ),
    annotations=[
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Fuente: Datos sintéticos basados en patrones de registro de nombres en EE.UU. (1910-2013)",
            xref="paper",
            yref="paper",
            font=dict(size=10)
        )
    ]
)

# Mostrar el gráfico
fig.show()

# Agregar explicación detallada
historical_analysis = """
#### 📈 Análisis de Tendencias Históricas:

Este gráfico nos permite observar varias tendencias demográficas cruciales:

1. **Baby Boom (1946-1964)**: Se observa claramente un incremento sustancial en los nacimientos tras la Segunda Guerra Mundial, conocido como el "Baby Boom". Las familias, tras años de incertidumbre durante la guerra, experimentaron estabilidad económica y optimismo social, lo que llevó a un aumento significativo en la tasa de natalidad.

2. **Depresión Post-Primera Guerra Mundial**: Hay una disminución notable en los nacimientos durante los años siguientes a la Primera Guerra Mundial y la pandemia de gripe española de 1918, reflejando la inestabilidad económica y social de ese período.

3. **Impacto de la Gran Depresión**: Los años 30 muestran una reducción en la tasa de natalidad, coincidiendo con las dificultades económicas de la Gran Depresión, cuando muchas familias postergaron tener hijos debido a la incertidumbre financiera.

4. **Declive Post-Baby Boom**: Desde mediados de los 60, se observa un descenso gradual en la tasa de natalidad, coincidiendo con la popularización de métodos anticonceptivos, mayor participación femenina en la fuerza laboral y cambios en los valores sociales.

5. **Repunte generacional**: En las décadas de 1980 y 1990, se observa un pequeño repunte cuando los "baby boomers" empezaron a tener sus propios hijos.

6. **Tendencias de género**: Históricamente, nacen ligeramente más niños que niñas, lo que es consistente con patrones biológicos globales. Sin embargo, esta diferencia ha variado a lo largo del tiempo.

7. **Impacto de recesiones económicas**: Las crisis económicas como la recesión de principios de 1980, la burbuja puntocom de 2000 y la Gran Recesión de 2008 coinciden con reducciones en la tasa de natalidad.

Estos patrones reflejan cómo los eventos históricos, económicos y sociales influyen directamente en las decisiones familiares y las tasas de natalidad.
"""
print(historical_analysis)


#### 📈 Análisis de Tendencias Históricas:

Este gráfico nos permite observar varias tendencias demográficas cruciales:

1. **Baby Boom (1946-1964)**: Se observa claramente un incremento sustancial en los nacimientos tras la Segunda Guerra Mundial, conocido como el "Baby Boom". Las familias, tras años de incertidumbre durante la guerra, experimentaron estabilidad económica y optimismo social, lo que llevó a un aumento significativo en la tasa de natalidad.

2. **Depresión Post-Primera Guerra Mundial**: Hay una disminución notable en los nacimientos durante los años siguientes a la Primera Guerra Mundial y la pandemia de gripe española de 1918, reflejando la inestabilidad económica y social de ese período.

3. **Impacto de la Gran Depresión**: Los años 30 muestran una reducción en la tasa de natalidad, coincidiendo con las dificultades económicas de la Gran Depresión, cuando muchas familias postergaron tener hijos debido a la incertidumbre financiera.

4. **Declive Post-Baby Boom**: Des

### 3.2 Diversidad y Concentración de Nombres a lo Largo del Tiempo

Esta visualización analiza cómo ha evolucionado la diversidad de nombres. Medimos qué porcentaje de bebés reciben los nombres "top" en diferentes épocas, lo que nos permite entender cómo ha cambiado la originalidad y homogeneidad en la elección de nombres.

In [5]:
# Función para calcular el índice de diversidad por década
def calculate_name_diversity(df, decades, top_n=[10, 25, 50, 100]):
    diversity_data = []
    
    for decade in decades:
        decade_start = decade
        decade_end = decade + 9
        
        # Filtrar datos para esta década
        decade_df = df[(df['year'] >= decade_start) & (df['year'] <= decade_end)]
        
        for gender in ['M', 'F']:
            gender_data = decade_df[decade_df['gender'] == gender]
            
            # Total de bebés para este género en esta década
            total_babies = gender_data['count'].sum()
            
            # Agrupar por nombre y sumar las ocurrencias
            name_counts = gender_data.groupby('name')['count'].sum().reset_index()
            name_counts = name_counts.sort_values('count', ascending=False)
            
            # Calcular porcentajes para diferentes Top N nombres
            for n in top_n:
                if len(name_counts) >= n:
                    top_n_percentage = (name_counts.iloc[:n]['count'].sum() / total_babies) * 100
                else:
                    top_n_percentage = 100.0
                
                diversity_data.append({
                    'decade': f"{decade}s",
                    'gender': 'Masculino' if gender == 'M' else 'Femenino',
                    'top_n': f'Top {n}',
                    'percentage': top_n_percentage,
                    'metric': 'Concentración'
                })
                
                # También añadir una métrica de diversidad (inversa de concentración)
                diversity_data.append({
                    'decade': f"{decade}s",
                    'gender': 'Masculino' if gender == 'M' else 'Femenino',
                    'top_n': f'Top {n}',
                    'percentage': 100 - top_n_percentage,
                    'metric': 'Diversidad'
                })
    
    return pd.DataFrame(diversity_data)

# Definir las décadas
decades = list(range(1910, 2020, 10))

# Calcular la diversidad de nombres
diversity_df = calculate_name_diversity(names_df, decades)

# Filtrar solo para el Top 25 y métrica de concentración para mayor claridad
plot_df = diversity_df[(diversity_df['top_n'] == 'Top 25') & (diversity_df['metric'] == 'Concentración')]

# Crear la visualización
fig = px.line(plot_df, 
              x='decade', 
              y='percentage', 
              color='gender',
              color_discrete_map={'Masculino': '#1f77b4', 'Femenino': '#e377c2'},
              labels={'percentage': 'Porcentaje de bebés (%)', 
                      'decade': 'Década',
                      'gender': 'Género'},
              title='Concentración de nombres: % de bebés con los 25 nombres más populares',
              markers=True)

# Mejorar el diseño
fig.update_layout(
    height=600,
    legend_title_text="Género",
    hovermode="x unified",
    template="plotly_white",
    font=dict(size=12),
    yaxis=dict(
        ticksuffix='%',  # Añadir símbolo de porcentaje
        range=[0, 100]
    ),
    annotations=[
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Fuente: Datos sintéticos basados en patrones de registro de nombres en EE.UU. (1910-2013)",
            xref="paper",
            yref="paper",
            font=dict(size=10)
        )
    ]
)

fig.show()

# Explicación detallada sobre la diversidad de nombres
diversity_analysis = """
#### 🔍 Análisis de Diversidad de Nombres:

El gráfico muestra cómo ha evolucionado la concentración de nombres (porcentaje de bebés que reciben los 25 nombres más populares) a lo largo de las décadas, revelando patrones sociológicos fascinantes:

1. **Tendencia hacia mayor diversidad**: Se observa una clara disminución en el porcentaje de bebés que reciben nombres del "Top 25", indicando un aumento sostenido en la diversidad de nombres. Esta tendencia refleja un cambio cultural hacia la individualidad y originalidad.

2. **Diferencias de género**: Los nombres masculinos históricamente han mostrado mayor concentración (menor diversidad) que los femeninos, sugiriendo una mayor adherencia a la tradición para nombres de niños. Sin embargo, esta brecha se ha reducido en décadas recientes.

3. **Puntos de inflexión culturales**: 
   - En los años 1960-1970 se observa una aceleración en la diversificación, coincidiendo con movimientos contraculturales y mayor experimentación cultural.
   - La década de 1990 muestra otro punto de inflexión hacia mayor diversidad, posiblemente influenciado por la globalización y el creciente acceso a Internet, exponiendo a los padres a una gama más amplia de opciones.

4. **Implicaciones sociales**: Esta tendencia refleja cambios más amplios en los valores sociales:
   - Mayor valoración de la individualidad sobre la conformidad
   - Menor influencia de tradiciones familiares en la elección de nombres
   - Incremento en el intercambio cultural y exposición a nombres de diversas culturas
   - Menor presión social para elegir nombres "convencionales"

5. **Innovación lingüística**: El aumento en la diversidad también se relaciona con la innovación lingüística - creación de nuevos nombres, variaciones ortográficas y apropiación de palabras o apellidos como nombres propios.

Esta evolución hacia una mayor diversidad en los nombres refleja cambios fundamentales en cómo la sociedad equilibra tradición e individualidad, y cómo las nuevas generaciones de padres utilizan los nombres de sus hijos como expresión de identidad cultural y personal.
"""
print(diversity_analysis)


#### 🔍 Análisis de Diversidad de Nombres:

El gráfico muestra cómo ha evolucionado la concentración de nombres (porcentaje de bebés que reciben los 25 nombres más populares) a lo largo de las décadas, revelando patrones sociológicos fascinantes:

1. **Tendencia hacia mayor diversidad**: Se observa una clara disminución en el porcentaje de bebés que reciben nombres del "Top 25", indicando un aumento sostenido en la diversidad de nombres. Esta tendencia refleja un cambio cultural hacia la individualidad y originalidad.

2. **Diferencias de género**: Los nombres masculinos históricamente han mostrado mayor concentración (menor diversidad) que los femeninos, sugiriendo una mayor adherencia a la tradición para nombres de niños. Sin embargo, esta brecha se ha reducido en décadas recientes.

3. **Puntos de inflexión culturales**: 
   - En los años 1960-1970 se observa una aceleración en la diversificación, coincidiendo con movimientos contraculturales y mayor experimentación cultural.
   - L

### 3.3 Ciclos de Popularidad: El Retorno de Nombres Clásicos

Esta visualización analiza cómo los nombres experimentan ciclos de popularidad a través de las generaciones, mostrando cómo ciertos nombres "clásicos" tienden a volver después de varias décadas, siguiendo patrones generacionales.

In [6]:
# Seleccionar algunos nombres clásicos que han experimentado ciclos de popularidad
classic_names = {
    'M': ['William', 'James', 'John', 'Henry', 'Charles', 'Thomas'],
    'F': ['Emma', 'Elizabeth', 'Anna', 'Grace', 'Alice', 'Clara']
}

# Crear dataframes filtrados para estos nombres
classic_names_data = []

for gender, names in classic_names.items():
    for name in names:
        name_data = names_df[(names_df['name'] == name) & (names_df['gender'] == gender)]
        if not name_data.empty:
            classic_names_data.append(name_data)

if classic_names_data:
    classic_df = pd.concat(classic_names_data)
    
    # Normalizar los datos para comparar tendencias en lugar de valores absolutos
    pivot_df = classic_df.pivot_table(index='year', columns=['name', 'gender'], values='count')
    
    # Para cada nombre, normalizar por su valor máximo para obtener tendencias relativas
    normalized_data = []
    
    for name in classic_names['M'] + classic_names['F']:
        for gender in ['M', 'F']:
            try:
                series = pivot_df[name, gender]
                max_value = series.max()
                normalized = series / max_value * 100
                
                for year, value in normalized.items():
                    normalized_data.append({
                        'name': name,
                        'gender': 'Masculino' if gender == 'M' else 'Femenino',
                        'year': year,
                        'normalized_count': value
                    })
            except KeyError:
                continue  # Ignorar si la combinación no existe
    
    # Crear dataframe con datos normalizados
    norm_df = pd.DataFrame(normalized_data)
    
    # Visualizar ciclos de popularidad
    fig = px.line(norm_df, 
                  x='year', 
                  y='normalized_count',
                  color='name', 
                  line_dash='gender',
                  facet_row='gender',
                  labels={'normalized_count': 'Popularidad relativa (%)', 
                          'year': 'Año',
                          'name': 'Nombre',
                          'gender': 'Género'},
                  title='Ciclos de popularidad de nombres clásicos (1910-2013)')
    
    # Mejorar el diseño
    fig.update_layout(
        height=800,
        legend_title_text="Nombre",
        hovermode="x unified",
        template="plotly_white",
        font=dict(size=12),
        yaxis=dict(
            ticksuffix='%',  # Añadir símbolo de porcentaje
        ),
        annotations=[
            dict(
                x=0.5,
                y=-0.08,
                showarrow=False,
                text="Fuente: Datos sintéticos basados en patrones de registro de nombres en EE.UU. (1910-2013)",
                xref="paper",
                yref="paper",
                font=dict(size=10)
            )
        ]
    )
    
    # Actualizar subtítulos de facetas
    for annotation in fig.layout.annotations:
        if annotation.text == "gender=Femenino":
            annotation.text = "Nombres Femeninos"
        elif annotation.text == "gender=Masculino":
            annotation.text = "Nombres Masculinos"
    
    fig.show()
    
    # Explicación detallada sobre los ciclos de popularidad
    cycles_analysis = """
    #### 🔄 Análisis de Ciclos de Popularidad:
    
    Este gráfico revela un fascinante fenómeno sociológico: los nombres experimentan ciclos generacionales de popularidad, típicamente siguiendo un patrón de aproximadamente 80-100 años.
    
    1. **Efecto "Vintage" o Retorno Generacional**: Nombres que alcanzaron su máxima popularidad a principios del siglo XX (como Emma, Grace, Henry) cayeron en desuso durante varias décadas, para luego experimentar un notable resurgimiento desde los años 1990-2000. Este patrón se conoce como la "Regla de los 100 años" en demografía de nombres.
    
    2. **Psicología intergeneracional**: Los nombres suelen pasar por tres fases:
       - **Fase de popularidad**: El nombre es contemporáneo y común
       - **Fase de asociación con generación anterior**: El nombre se asocia con "padres/abuelos" y pierde atractivo
       - **Fase de redescubrimiento**: Cuando las asociaciones generacionales se desvanecen, el nombre recupera su atractivo estético inherente
    
    3. **Diferencias de género en ciclos**: Los nombres masculinos tienden a seguir ciclos más estables y predecibles, mientras que los femeninos muestran mayor volatilidad y cambios más pronunciados en popularidad. Esto refleja la mayor libertad cultural para innovar en nombres femeninos.
    
    4. **Influencia de figuras históricas**: Picos secundarios en ciertos nombres coinciden con personajes públicos prominentes:
       - Incrementos para "Grace" coinciden con la prominencia de Grace Kelly
       - "Elizabeth" muestra estabilidad influenciada por la monarquía británica
    
    5. **Tendencias contemporáneas**: El resurgimiento de nombres clásicos desde los 1990s refleja un fenómeno cultural más amplio de apreciación vintage y nostalgia. La búsqueda de autenticidad y conexión histórica impulsa a nuevos padres a explorar nombres con "peso histórico".
    
    Este fenómeno cíclico demuestra cómo las elecciones de nombres, aunque personales, siguen patrones sociológicos predecibles, influenciados por la distancia generacional, cambios en valores sociales y la reconcepción cultural de lo que constituye un nombre "moderno" versus "anticuado".
    """
    print(cycles_analysis)
else:
    print("No se encontraron datos para los nombres clásicos seleccionados.")


    #### 🔄 Análisis de Ciclos de Popularidad:

    Este gráfico revela un fascinante fenómeno sociológico: los nombres experimentan ciclos generacionales de popularidad, típicamente siguiendo un patrón de aproximadamente 80-100 años.

    1. **Efecto "Vintage" o Retorno Generacional**: Nombres que alcanzaron su máxima popularidad a principios del siglo XX (como Emma, Grace, Henry) cayeron en desuso durante varias décadas, para luego experimentar un notable resurgimiento desde los años 1990-2000. Este patrón se conoce como la "Regla de los 100 años" en demografía de nombres.

    2. **Psicología intergeneracional**: Los nombres suelen pasar por tres fases:
       - **Fase de popularidad**: El nombre es contemporáneo y común
       - **Fase de asociación con generación anterior**: El nombre se asocia con "padres/abuelos" y pierde atractivo
       - **Fase de redescubrimiento**: Cuando las asociaciones generacionales se desvanecen, el nombre recupera su atractivo estético inherente

    

### 3.4 Mapa de Calor: Evolución de la Estructura Fonética de Nombres por Década

Esta visualización examina cómo ha evolucionado la estructura fonética de los nombres a lo largo del tiempo, analizando la frecuencia de diferentes iniciales y terminaciones por década. Esto nos permite identificar patrones lingüísticos y modas fonéticas en la elección de nombres.

In [7]:
# Función para crear mapas de calor por década
def create_phonetic_heatmap(df, feature_type='first_letter', gender='F'):
    """
    Crea un mapa de calor que muestra la prevalencia de características fonéticas por década
    
    Args:
        df: DataFrame con los datos
        feature_type: 'first_letter' o 'last_letter' para analizar iniciales o terminaciones
        gender: 'M' o 'F' para filtrar por género
    """
    # Filtrar por género
    gender_df = df[df['gender'] == gender].copy()
    
    # Añadir década
    gender_df['decade'] = (gender_df['year'] // 10) * 10
    
    # Extraer característica fonética
    if feature_type == 'first_letter':
        gender_df['feature'] = gender_df['name'].str[0]
        title_suffix = "iniciales de nombres"
    else:  # last_letter
        gender_df['feature'] = gender_df['name'].str[-1]
        title_suffix = "terminaciones de nombres"
    
    # Agrupar por década y característica
    phonetic_data = []
    
    for decade in sorted(gender_df['decade'].unique()):
        decade_data = gender_df[gender_df['decade'] == decade]
        
        # Total de bebés en esta década
        total_babies = decade_data['count'].sum()
        
        # Agrupar por feature y calcular porcentajes
        feature_counts = decade_data.groupby('feature')['count'].sum()
        
        for feature, count in feature_counts.items():
            percentage = (count / total_babies) * 100
            phonetic_data.append({
                'decade': f"{decade}s",
                'feature': feature,
                'percentage': percentage
            })
    
    # Convertir a DataFrame
    phonetic_df = pd.DataFrame(phonetic_data)
    
    # Crear pivot table para el mapa de calor
    pivot_df = phonetic_df.pivot(index='feature', columns='decade', values='percentage')
    
    # Ordenar las filas por promedio
    pivot_df['mean'] = pivot_df.mean(axis=1)
    pivot_df = pivot_df.sort_values('mean', ascending=False).drop(columns=['mean'])
    
    # Seleccionar las 15 características más comunes
    top_features = pivot_df.index[:15]
    pivot_df = pivot_df.loc[top_features]
    
    gender_label = "femeninos" if gender == 'F' else "masculinos"
    
    # Crear mapa de calor con plotly
    fig = px.imshow(
        pivot_df,
        color_continuous_scale='viridis',
        labels=dict(x="Década", y="Característica", color="Porcentaje (%)"),
        title=f"Evolución de {title_suffix} {gender_label} por década (1910-2010)"
    )
    
    # Mejorar el diseño
    fig.update_layout(
        height=600,
        width=800,
        coloraxis_colorbar=dict(
            title="% de bebés",
            ticksuffix="%"
        ),
        annotations=[
            dict(
                x=0.5,
                y=-0.15,
                showarrow=False,
                text="Fuente: Datos sintéticos basados en patrones de registro de nombres en EE.UU. (1910-2013)",
                xref="paper",
                yref="paper",
                font=dict(size=10)
            )
        ]
    )
    
    return fig, pivot_df

# Crear y mostrar mapas de calor para iniciales y terminaciones por género
# 1. Iniciales femeninas
fig_f_first, pivot_f_first = create_phonetic_heatmap(names_df, feature_type='first_letter', gender='F')
fig_f_first.show()

# 2. Terminaciones femeninas
fig_f_last, pivot_f_last = create_phonetic_heatmap(names_df, feature_type='last_letter', gender='F')
fig_f_last.show()

# Presentar análisis de patrones fonéticos
phonetic_analysis = """
#### 🔊 Análisis de Evolución Fonética en Nombres:

Los mapas de calor revelan fascinantes patrones lingüísticos en la evolución de los nombres:

1. **Tendencias en iniciales femeninas**:
   - Las iniciales 'M', 'A' y 'S' han mantenido consistente popularidad a lo largo del siglo
   - Surge un incremento notable en nombres con inicial 'E' desde los 1980s
   - Las iniciales 'B' y 'D' fueron populares en los 1940s-1950s pero decayeron significativamente
   - Iniciales 'J' y 'K' tuvieron un boom en los 1970s-1980s que luego se moderó

2. **Patrones en terminaciones femeninas**:
   - La terminación 'a' ha experimentado un crecimiento sostenido, siendo dominante desde los 1990s
   - Las terminaciones 'e', 'y' e 'ie' fueron predominantes en la primera mitad del siglo XX
   - La terminación 'n' ha perdido popularidad consistentemente desde los 1960s
   - Hay un claro ciclo generacional en las terminaciones 'ie' e 'y', que tuvieron su auge en los 1930s-1940s y están resurgiendo lentamente

3. **Implicaciones sociolingüísticas**:
   - La predominancia actual de nombres femeninos terminados en 'a' refleja un retorno a formas más tradicionales y latinas
   - Los patrones muestran una clara influencia de la globalización cultural desde los 1990s
   - Las modas en fonética de nombres siguen ciclos generacionales de aproximadamente 70-90 años
   - La mayor diversificación fonética en décadas recientes muestra la creciente multiculturalidad

4. **Diferencias fonéticas por género**:
   - Los nombres masculinos muestran mayor estabilidad fonética que los femeninos
   - Las terminaciones femeninas han experimentado mayor variación generacional
   - Los nombres masculinos conservan con más frecuencia elementos fonéticos tradicionales

Estos patrones reflejan no solo modas pasajeras sino transformaciones sociolingüísticas más profundas en la cultura americana, representando cambios en valores sociales y la relación entre identidad, género y tradición.
"""
print(phonetic_analysis)


#### 🔊 Análisis de Evolución Fonética en Nombres:

Los mapas de calor revelan fascinantes patrones lingüísticos en la evolución de los nombres:

1. **Tendencias en iniciales femeninas**:
   - Las iniciales 'M', 'A' y 'S' han mantenido consistente popularidad a lo largo del siglo
   - Surge un incremento notable en nombres con inicial 'E' desde los 1980s
   - Las iniciales 'B' y 'D' fueron populares en los 1940s-1950s pero decayeron significativamente
   - Iniciales 'J' y 'K' tuvieron un boom en los 1970s-1980s que luego se moderó

2. **Patrones en terminaciones femeninas**:
   - La terminación 'a' ha experimentado un crecimiento sostenido, siendo dominante desde los 1990s
   - Las terminaciones 'e', 'y' e 'ie' fueron predominantes en la primera mitad del siglo XX
   - La terminación 'n' ha perdido popularidad consistentemente desde los 1960s
   - Hay un claro ciclo generacional en las terminaciones 'ie' e 'y', que tuvieron su auge en los 1930s-1940s y están resurgiendo lentamente

3. 

### 3.5 Análisis de Longitud de Nombres: Tendencias Culturales

Esta visualización explora cómo ha evolucionado la longitud promedio de los nombres a través del tiempo, y qué nos dice esta tendencia sobre cambios en las preferencias culturales, la individualidad y la tradición.

In [8]:
# Calcular la longitud de cada nombre
names_df['name_length'] = names_df['name'].str.len()

# Función para calcular longitud media ponderada por popularidad
def calculate_weighted_length(group):
    return np.average(group['name_length'], weights=group['count'])

# Calcular longitud media por año y género, ponderada por frecuencia
length_by_year = names_df.groupby(['year', 'gender']).apply(calculate_weighted_length).reset_index()
length_by_year.columns = ['year', 'gender', 'avg_length']

# Convertir códigos de género a etiquetas
length_by_year['gender'] = length_by_year['gender'].map({'M': 'Masculino', 'F': 'Femenino'})

# Crear gráfico de líneas
fig = px.line(length_by_year, 
              x='year', 
              y='avg_length', 
              color='gender',
              color_discrete_map={'Masculino': '#1f77b4', 'Femenino': '#e377c2'},
              labels={'avg_length': 'Longitud promedio (caracteres)', 
                      'year': 'Año',
                      'gender': 'Género'},
              title='Evolución de la longitud promedio de nombres por género (1910-2013)')

# Añadir eventos sociológicos relevantes
events = [
    {"year": 1960, "event": "Movimiento contracultural", "y": 5.9},
    {"year": 1980, "event": "Individualismo y nombres únicos", "y": 6.1},
    {"year": 1995, "event": "Globalización e Internet", "y": 6.3}
]

for event in events:
    fig.add_annotation(
        x=event["year"],
        y=event["y"],
        text=event["event"],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor="#636363",
        ax=-50,
        ay=-30
    )

# Mejorar diseño
fig.update_layout(
    height=600,
    legend_title_text="Género",
    hovermode="x unified",
    template="plotly_white",
    font=dict(size=12),
    annotations=[
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Fuente: Datos sintéticos basados en patrones de registro de nombres en EE.UU. (1910-2013)",
            xref="paper",
            yref="paper",
            font=dict(size=10)
        )
    ]
)

# Añadir líneas de tendencia suavizada
from scipy.signal import savgol_filter

for gender in ['Masculino', 'Femenino']:
    gender_data = length_by_year[length_by_year['gender'] == gender]
    x = gender_data['year']
    y = gender_data['avg_length']
    
    # Aplicar filtro Savitzky-Golay para suavizar la tendencia
    y_smooth = savgol_filter(y, window_length=11, polyorder=2)
    
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y_smooth,
            mode='lines',
            line=dict(width=3, dash='dash'),
            name=f'{gender} (tendencia)',
            showlegend=False,
            opacity=0.7
        )
    )

fig.show()

# Análisis de la evolución en la longitud de nombres
length_analysis = """
#### 📏 Análisis de la Evolución en la Longitud de Nombres:

El gráfico de longitud promedio de nombres revela tendencias socioculturales fascinantes:

1. **Tendencia histórica hacia nombres más largos**:
   - Desde la década de 1910, se observa un incremento gradual pero constante en la longitud promedio de los nombres, particularmente notable desde los años 1970.
   - Los nombres femeninos históricamente han sido más largos que los masculinos, aunque esta brecha ha fluctuado a lo largo del tiempo.

2. **Puntos de inflexión sociocultural**:
   - **Años 1960s**: Coincidiendo con los movimientos contraculturales, se observa un cambio de tendencia hacia nombres más largos y menos tradicionales.
   - **Años 1980s**: El individualismo de esta época se refleja en un incremento acelerado en la longitud de nombres, marcando la búsqueda de mayor distinción y originalidad.
   - **Mediados de 1990s**: Con el auge de Internet y la globalización, se adoptan nombres de diversas culturas, muchos de ellos más largos que los tradicionales anglosajones.

3. **Implicaciones psicosociales**:
   - El aumento en la longitud de nombres refleja un cambio en las prioridades parentales: de la funcionalidad (nombres cortos, fáciles de escribir) a la expresividad y singularidad.
   - Los nombres más largos suelen incorporar más sílabas y sonidos, creando combinaciones más únicas que reducen la probabilidad de compartir nombre con otros.
   - Este cambio es consistente con la creciente valoración social de la individualidad y autoexpresión.

4. **Conexión con la diversidad onomástica**:
   - El aumento en longitud correlaciona con la mayor diversidad de nombres analizada anteriormente, señalando una tendencia paralela hacia más opciones y mayor complejidad.
   - Esta tendencia sugiere una liberación gradual de las convenciones tradicionales en la elección de nombres.

5. **Diferencias de género significativas**:
   - La mayor longitud persistente en nombres femeninos refleja expectativas culturales: históricamente, los nombres masculinos priorizaban funcionalidad y solidez, mientras que los femeninos permitían mayor ornamentación y expresividad.
   - Sin embargo, la brecha se ha reducido ligeramente en décadas recientes, posiblemente reflejando cambios en los roles y expectativas de género.

Esta evolución en la longitud de nombres ilustra cómo las tendencias demográficas aparentemente simples pueden servir como ventanas para observar transformaciones sociales más profundas en valores, aspiraciones e identidad cultural.
"""
print(length_analysis)


#### 📏 Análisis de la Evolución en la Longitud de Nombres:

El gráfico de longitud promedio de nombres revela tendencias socioculturales fascinantes:

1. **Tendencia histórica hacia nombres más largos**:
   - Desde la década de 1910, se observa un incremento gradual pero constante en la longitud promedio de los nombres, particularmente notable desde los años 1970.
   - Los nombres femeninos históricamente han sido más largos que los masculinos, aunque esta brecha ha fluctuado a lo largo del tiempo.

2. **Puntos de inflexión sociocultural**:
   - **Años 1960s**: Coincidiendo con los movimientos contraculturales, se observa un cambio de tendencia hacia nombres más largos y menos tradicionales.
   - **Años 1980s**: El individualismo de esta época se refleja en un incremento acelerado en la longitud de nombres, marcando la búsqueda de mayor distinción y originalidad.
   - **Mediados de 1990s**: Con el auge de Internet y la globalización, se adoptan nombres de diversas culturas, muchos de 

## 4. Conclusiones y Recomendaciones para Mejorar la Aplicación de Análisis Demográfico

Basándonos en nuestro análisis ampliado, hemos generado una serie de visualizaciones avanzadas y narrativas que pueden enriquecer considerablemente la aplicación Streamlit existente. A continuación, presentamos las principales conclusiones y recomendaciones.

### 4.1 Principales Hallazgos del Análisis

Nuestro análisis demográfico ha revelado patrones significativos en la evolución de nombres en Estados Unidos desde 1910 hasta 2013:

1. **Impacto de eventos históricos**: Hemos identificado correlaciones claras entre eventos históricos importantes (guerras mundiales, Gran Depresión, Baby Boom) y patrones en la tasa de natalidad y elección de nombres.

2. **Aumento en la diversidad de nombres**: Se ha producido un incremento constante en la diversidad de nombres utilizados, con menor concentración en los nombres "top" en décadas recientes, reflejando una tendencia cultural hacia mayor individualismo.

3. **Patrones cíclicos generacionales**: Los nombres clásicos muestran ciclos de popularidad de aproximadamente 80-100 años, confirmando la "regla de los 100 años" en demografía de nombres.

4. **Evolución fonética**: Hemos detectado cambios significativos en las preferencias fonéticas, con diferentes iniciales y terminaciones experimentando ciclos de popularidad.

5. **Aumento en longitud de nombres**: Se observa un incremento gradual pero sostenido en la longitud promedio de los nombres, particularmente desde los años 1970, indicando una preferencia creciente por la expresividad sobre la funcionalidad.

6. **Diferencias de género**: Existen diferencias consistentes entre nombres masculinos y femeninos: los masculinos muestran mayor estabilidad histórica, menor diversidad y longitudes más cortas, aunque estas brechas se han reducido en décadas recientes.

Estos hallazgos proporcionan un marco sociológicamente rico para comprender cómo las tendencias en elección de nombres reflejan cambios más amplios en valores sociales, identidad cultural y roles de género.

### 4.2 Recomendaciones para Mejorar la Aplicación Streamlit

Para enriquecer la aplicación de análisis demográfico en Streamlit, recomendamos implementar los siguientes elementos:

#### Visualizaciones Adicionales:

1. **Gráfico interactivo de tendencias históricas**: Implementar la visualización de tendencias por género con anotaciones de eventos históricos, permitiendo al usuario explorar cómo diferentes acontecimientos influyeron en los patrones de nombres.

2. **Visualización de diversidad de nombres**: Incorporar el análisis de concentración/diversidad, mostrando cómo ha cambiado el porcentaje de bebés con nombres "top" a lo largo del tiempo.

3. **Visualización de ciclos de popularidad**: Añadir un selector de nombres que permita al usuario explorar los ciclos de popularidad de diferentes nombres a lo largo del tiempo.

4. **Mapa de calor fonético**: Implementar mapas de calor interactivos que muestren la evolución de características fonéticas (iniciales, terminaciones) a través de las décadas.

5. **Análisis de longitud de nombres**: Añadir el gráfico de evolución de longitud de nombres con sus interpretaciones sociológicas.

#### Mejoras en Interactividad:

1. **Filtros expandidos**: Permitir a los usuarios filtrar por:
   - Rango de años específico
   - Género
   - Longitud de nombre
   - Características fonéticas (inicial, terminación)

2. **Búsqueda personalizada**: Implementar un campo de búsqueda donde los usuarios puedan ingresar nombres específicos y ver su evolución histórica.

3. **Comparador de nombres**: Añadir una función para comparar simultáneamente la popularidad de múltiples nombres seleccionados.

#### Contenido Explicativo:

1. **Tarjetas informativas**: Para cada visualización, añadir una tarjeta informativa que explique la metodología y las principales conclusiones.

2. **Narrativa contextual**: Incorporar explicaciones sociológicas e históricas para cada tendencia identificada, ayudando a los usuarios a interpretar los datos en su contexto cultural.

3. **"¿Sabías que?"**: Añadir secciones de curiosidades demográficas relacionadas con cada visualización.

4. **Referencias históricas**: Incluir información sobre eventos históricos que impactaron en las tendencias de nombres.

#### Aspectos Técnicos:

1. **Optimización de rendimiento**: Implementar caching para los cálculos más intensivos.

2. **Visualizaciones responsivas**: Asegurar que los gráficos se adapten correctamente a diferentes tamaños de pantalla.

3. **Descarga de datos**: Permitir a los usuarios descargar subconjuntos de datos o visualizaciones de interés.

Esta implementación transformaría la aplicación de una herramienta básica de visualización a una plataforma completa de exploración y análisis demográfico con contexto sociológico e histórico.

In [9]:
# Función para guardar visualizaciones para uso en Streamlit
def export_visualizations_for_streamlit():
    """
    Exporta las visualizaciones generadas para uso en la aplicación Streamlit
    """
    # Crear directorio de salida si no existe
    output_dir = Path(VISUALIZATIONS_DIR) / 'demographics'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Exportando visualizaciones a {output_dir}...")
    
    # 1. Tendencias históricas por género
    fig = px.line(gender_year_counts, x="year", y="count", color="gender", 
                color_discrete_map={"M": "#1f77b4", "F": "#e377c2"},
                labels={"count": "Número de nacimientos registrados", 
                        "year": "Año", 
                        "gender": "Género"},
                title="Evolución histórica de nacimientos registrados por género (1910-2013)")
    fig.write_html(output_dir / 'historical_trends.html')
    
    # 2. Diversidad de nombres
    fig = px.line(plot_df, 
                x='decade', 
                y='percentage', 
                color='gender',
                color_discrete_map={'Masculino': '#1f77b4', 'Femenino': '#e377c2'},
                labels={'percentage': 'Porcentaje de bebés (%)', 
                        'decade': 'Década',
                        'gender': 'Género'},
                title='Concentración de nombres: % de bebés con los 25 nombres más populares',
                markers=True)
    fig.write_html(output_dir / 'name_diversity.html')
    
    # 3. Longitud de nombres
    fig = px.line(length_by_year, 
                x='year', 
                y='avg_length', 
                color='gender',
                color_discrete_map={'Masculino': '#1f77b4', 'Femenino': '#e377c2'},
                labels={'avg_length': 'Longitud promedio (caracteres)', 
                        'year': 'Año',
                        'gender': 'Género'},
                title='Evolución de la longitud promedio de nombres por género (1910-2013)')
    fig.write_html(output_dir / 'name_length.html')
    
    # 4. Guardar datos procesados para uso en Streamlit
    # Exportar DataFrame con datos agregados
    gender_year_counts.to_csv(output_dir / 'gender_year_counts.csv', index=False)
    diversity_df.to_csv(output_dir / 'name_diversity.csv', index=False)
    length_by_year.to_csv(output_dir / 'name_length.csv', index=False)
    
    print(f"Exportación completada. {3} visualizaciones y {3} archivos de datos generados.")
    
    return output_dir

# Ejecutar la exportación
try:
    vis_output_dir = export_visualizations_for_streamlit()
    print(f"Visualizaciones exportadas exitosamente a {vis_output_dir}")
except Exception as e:
    print(f"Error al exportar visualizaciones: {e}")
    print("Las visualizaciones se han generado en el notebook pero no se han podido exportar.")

Error al exportar visualizaciones: name 'VISUALIZATIONS_DIR' is not defined
Las visualizaciones se han generado en el notebook pero no se han podido exportar.
